# The 6 Fs -- IFS's Core Algorithm

The **6 Fs** is the standard IFS protocol for engaging with Protectors (Managers and Firefighters). It is a sequential state machine -- the "game loop" of the therapeutic process. Each step must complete before the next begins.

| Step | Name | Purpose |
|---:|---|---|
| 1 | **Find** | Locate the Part -- identify it through a trailhead (body sensation, image, thought, voice) |
| 2 | **Focus** | Direct attention to it -- select it as the target of engagement |
| 3 | **Flesh Out** | Gather metadata -- ask the Part about its age, appearance, beliefs |
| 4 | **Feel Toward** | **THE CRITICAL GATE** -- check if Self is present. If not, another Part has blended |
| 5 | **Befriend** | Build relationship -- express appreciation for the Part's protective role |
| 6 | **Fear** | Identify worst-case scenarios -- "What would happen if you stopped doing your job?" |

In this notebook we walk through all six steps on the **happy path** -- Self-energy is sufficient throughout. The next notebook explores what happens when the Feel-Toward gate fails.

In [1]:
from agentic_ifs import (
    Manager,
    Firefighter,
    Exile,
    Burden,
    BurdenType,
    ProtectionGraph,
    Edge,
    EdgeType,
    PolarizationEdge,
    SelfSystem,
    BlendState,
    SixFsStateMachine,
    SixFsStep,
    Trailhead,
    TrailheadLog,
    TrailheadType,
    FocusShift,
)

# Create Parts
manager = Manager(
    narrative="The Inner Critic -- formed at age 12 after school failure",
    age=12,
    intent="Keep us safe from criticism by being perfect first",
    triggers=["criticism from authority", "perceived failure"],
    strategies=["perfectionism", "over-preparation", "self-criticism"],
    rigidity=0.8,
)

firefighter = Firefighter(
    narrative="The Procrastinator -- shuts down when pressure is too high",
    age=14,
    intent="Protect from overwhelm by stopping all effort",
    pain_threshold=0.6,
    extinguishing_behaviors=["avoidance", "distraction", "numbing"],
)

exile = Exile(
    narrative="The Wounded Child -- carries shame from early school failure",
    age=7,
    intent="Hold the pain so the system can function",
    burden=Burden(
        burden_type=BurdenType.PERSONAL,
        origin="Age 7, school failure",
        content="I am not enough",
        emotional_charge=0.9,
    ),
    emotional_charge=0.7,
)

# Build the graph
graph = ProtectionGraph()
graph.add_part(manager)
graph.add_part(firefighter)
graph.add_part(exile)

graph.add_edge(Edge(source_id=manager.id, target_id=exile.id, edge_type=EdgeType.PROTECTS))
graph.add_edge(Edge(source_id=firefighter.id, target_id=exile.id, edge_type=EdgeType.PROTECTS))
graph.add_polarization(PolarizationEdge(part_a_id=manager.id, part_b_id=firefighter.id, tension_level=0.7))

# Self-energy is high (0.8) for the happy path
ss = SelfSystem(self_energy=0.8)
log = TrailheadLog()

# Create the state machine
workflow = SixFsStateMachine(graph=graph, self_system=ss, log=log)

print(f"Self-energy:  {ss.self_energy}")
print(f"Parts:        {len(graph.nodes)}")
print(f"Current step: {workflow.current_step}")

Self-energy:  0.8
Parts:        3
Current step: None


## Step 1: Find

The **Find** step begins with a *trailhead* -- a signal that a Part is present. Trailheads can be somatic (body sensation), visual (internal image), auditory (internal voice), or cognitive (a thought or belief).

The facilitator (Self) scans the system: "Where in your body do you notice something? What comes up when you think about that situation?"

In [2]:
# A somatic trailhead -- tightness in the chest
trailhead = Trailhead(
    trailhead_type=TrailheadType.SOMATIC,
    intensity=0.6,
    description="Tightness in chest when boss gives feedback",
    associated_part_id=manager.id,
)

result = workflow.find(trailhead)

print(f"Step:        {result.step}")
print(f"Target Part: {result.target_part_id}")
print(f"Next step:   {result.next_step}")
print(f"Notes:       {result.notes}")

Step:        SixFsStep.FIND
Target Part: a0b7c596-7433-4b47-b6da-11d0e2e4d483
Next step:   SixFsStep.FOCUS
Notes:       Trailhead: Tightness in chest when boss gives feedback (somatic)


## Step 2: Focus

**Focus** means turning toward the Part with intention. In IFS, the facilitator asks: "Can you focus on that Part? Can you find it in your body or in your mind's eye?"

Computationally, this is `SelectTarget(PartID)` -- making the Part the current focus of the state machine.

In [3]:
result = workflow.focus(manager.id)

print(f"Step:        {result.step}")
print(f"Target Part: {result.target_part_id}")
print(f"Next step:   {result.next_step}")

Step:        SixFsStep.FOCUS
Target Part: a0b7c596-7433-4b47-b6da-11d0e2e4d483
Next step:   SixFsStep.FLESH_OUT


## Step 3: Flesh Out

**Flesh Out** gathers metadata about the Part. The facilitator asks: "How old is this Part? What does it look like? What does it believe? Where do you feel it in your body?"

The state machine reads the Part's stored attributes and returns them as notes.

In [4]:
result = workflow.flesh_out(manager.id)

print(f"Step:        {result.step}")
print(f"Target Part: {result.target_part_id}")
print(f"Next step:   {result.next_step}")
print(f"Notes:       {result.notes}")

Step:        SixFsStep.FLESH_OUT
Target Part: a0b7c596-7433-4b47-b6da-11d0e2e4d483
Next step:   SixFsStep.FEEL_TOWARD
Notes:       Age: 12, Intent: Keep us safe from criticism by being perfect first


## Step 4: Feel Toward -- THE CRITICAL GATE

**Feel Toward** is the most important step in the 6 Fs. The facilitator asks: *"How do you feel toward this Part?"*

If the answer is curiosity, compassion, or openness -- that is Self. Proceed.

If the answer is frustration, anger, fear, or judgment -- **that feeling is itself a Part that has blended.** The facilitator must work with *that* Part first (recursive check).

The gate checks: is `self_energy > COMPASSION_THRESHOLD` (0.5)? If yes, Self is present. If no, another Part has taken the seat.

In [5]:
result = workflow.feel_toward(manager.id)

print(f"Step:            {result.step}")
print(f"Target Part:     {result.target_part_id}")
print(f"Next step:       {result.next_step}")
print(f"Unblend needed:  {result.unblend_required}")
print(f"Notes:           {result.notes}")
print(f"\nSelf-energy: {ss.self_energy} > 0.5 (threshold) => Gate passes!")

Step:            SixFsStep.FEEL_TOWARD
Target Part:     a0b7c596-7433-4b47-b6da-11d0e2e4d483
Next step:       SixFsStep.BEFRIEND
Unblend needed:  None
Notes:           Self-energy sufficient — Self is present

Self-energy: 0.8 > 0.5 (threshold) => Gate passes!


## Step 5: Befriend

**Befriend** is about building a relationship between Self and the Part. The facilitator says: "I see you. I appreciate what you're trying to do for us. You've been working so hard."

The Part's `trust_level` toward Self increases. This is a gradual process -- trust is earned, not assumed.

In [6]:
print(f"Trust BEFORE befriend: {manager.trust_level}")

result = workflow.befriend(manager.id)

print(f"Trust AFTER befriend:  {manager.trust_level}")
print(f"\nStep:        {result.step}")
print(f"Next step:   {result.next_step}")
print(f"Notes:       {result.notes}")

Trust BEFORE befriend: 0.5
Trust AFTER befriend:  0.6

Step:        SixFsStep.BEFRIEND
Next step:   SixFsStep.FEAR
Notes:       Trust updated to 0.60


## Step 6: Fear

**Fear** asks the critical question: *"What are you afraid would happen if you stopped doing your job?"*

The Part's fears reveal the Exile it protects and the burden it carries. This is where the Protector's logic becomes visible: "If I stop being perfect, we'll be rejected. If I stop planning, we'll fail. If I stop criticising, no one else will prepare us."

In V1, this step reports the Part's protective relationships (edges from this Part to others).

In [7]:
result = workflow.fear(manager.id)

print(f"Step:        {result.step}")
print(f"Target Part: {result.target_part_id}")
print(f"Next step:   {result.next_step}  <-- None means 6 Fs complete")
print(f"Notes:       {result.notes}")

Step:        SixFsStep.FEAR
Target Part: a0b7c596-7433-4b47-b6da-11d0e2e4d483
Next step:   None  <-- None means 6 Fs complete
Notes:       Protects 1 Part(s). Part intent: 'Keep us safe from criticism by being perfect first'


## The Session Log

Every step in the 6 Fs is recorded in the `SelfSystem.session_log`. This provides a timestamped audit trail of the therapeutic process.

In [8]:
print(f"{'Timestamp':<28} {'Event Type':<12} {'Description'}")
print("-" * 90)
for entry in ss.session_log:
    ts = entry.timestamp.strftime("%Y-%m-%d %H:%M:%S.%f")[:23]
    print(f"{ts:<28} {entry.event_type:<12} {entry.description}")

Timestamp                    Event Type   Description
------------------------------------------------------------------------------------------
2026-02-27 12:03:06.108      six_fs       FIND: Trailhead 'Tightness in chest when boss gives feedback' observed
2026-02-27 12:03:13.907      six_fs       FOCUS: Attention directed to Part
2026-02-27 12:03:23.389      six_fs       FLESH_OUT: Part aged 12, intent: 'Keep us safe from criticism by being perfect first'
2026-02-27 12:03:29.639      six_fs       FEEL_TOWARD: Self-energy sufficient (0.80 > 0.5)
2026-02-27 12:04:11.318      six_fs       BEFRIEND: Trust updated to 0.60
2026-02-27 12:04:14.144      six_fs       FEAR: Part protects 1 other Part(s)


## Trailhead Types

Trailheads can manifest through different sensory modalities. IFS recognises four:

- **Somatic** -- body sensation (tightness, pressure, pain)
- **Visual** -- internal image (a young child, a wall, a dark room)
- **Auditory** -- internal voice ("You're not good enough", "Don't do it")
- **Cognitive** -- thought or belief ("I have to be perfect", "Nobody cares")

In [9]:
# Create one of each TrailheadType
trailheads = [
    Trailhead(
        trailhead_type=TrailheadType.SOMATIC,
        intensity=0.6,
        description="Tightness in chest when boss gives feedback",
        associated_part_id=manager.id,
    ),
    Trailhead(
        trailhead_type=TrailheadType.VISUAL,
        intensity=0.5,
        description="Image of a small child hiding under a desk",
        associated_part_id=exile.id,
    ),
    Trailhead(
        trailhead_type=TrailheadType.AUDITORY,
        intensity=0.8,
        description="Voice saying 'You have to try harder'",
        associated_part_id=manager.id,
    ),
    Trailhead(
        trailhead_type=TrailheadType.COGNITIVE,
        intensity=0.4,
        description="Thought: 'What is the point of even trying?'",
        associated_part_id=firefighter.id,
    ),
]

# Add all to a TrailheadLog
trail_log = TrailheadLog()
for t in trailheads:
    trail_log.add(t)

print(f"Total trailheads: {len(trail_log.entries)}")

# Filter by type
for tt in TrailheadType:
    matches = trail_log.get_by_type(tt)
    print(f"\n{tt.value.upper()} trailheads ({len(matches)}):")
    for m in matches:
        print(f"  - [{m.intensity:.1f}] {m.description}")

Total trailheads: 4

SOMATIC trailheads (1):
  - [0.6] Tightness in chest when boss gives feedback

VISUAL trailheads (1):
  - [0.5] Image of a small child hiding under a desk

AUDITORY trailheads (1):
  - [0.8] Voice saying 'You have to try harder'

COGNITIVE trailheads (1):
  - [0.4] Thought: 'What is the point of even trying?'


## The U-Turn (FocusShift)

The **U-Turn** is the foundational pivot in IFS: shifting attention from the external trigger ("My boss criticised me") to the internal Part ("My Inner Critic is activated"). The `FocusShift` model captures this transition as a meta-tag on the session.

This is the moment when the work begins -- not by changing the external situation, but by turning inward.

In [10]:
u_turn = FocusShift(
    from_subject="My boss's criticism of the quarterly report",
    to_subject="My Inner Critic Part -- the tightness in my chest",
)

print(f"From: {u_turn.from_subject}")
print(f"To:   {u_turn.to_subject}")
print(f"At:   {u_turn.timestamp.strftime('%Y-%m-%d %H:%M:%S UTC')}")
print()
print("The U-Turn: from 'what happened to me' to 'what is happening inside me'.")

From: My boss's criticism of the quarterly report
To:   My Inner Critic Part -- the tightness in my chest
At:   2026-02-27 12:04:38 UTC

The U-Turn: from 'what happened to me' to 'what is happening inside me'.


## Summary

In this notebook we walked through the complete 6 Fs workflow on the happy path:

1. **Find** -- a somatic trailhead signals the Inner Critic
2. **Focus** -- attention directed to the Part
3. **Flesh Out** -- metadata gathered (age 12, intent: "Keep us safe...")
4. **Feel Toward** -- Self-energy sufficient (0.8 > 0.5), gate passes
5. **Befriend** -- trust incremented (0.5 -> 0.6)
6. **Fear** -- protective relationships revealed

We also explored trailhead types and the U-Turn (FocusShift).

**Next:** In [04_blending_dynamics.ipynb](04_blending_dynamics.ipynb), we explore what happens when the Feel-Toward gate *fails* -- blending mechanics, the occlusion model, and system metrics.